In [1]:
import gym
import ptan
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tensorboardX import SummaryWriter
from lib import dqn_model, common

In [2]:
class DuelingDQN(nn.Module):
    #convolution layer same as before
    def __init__(self, input_shape, n_actions):
        super(DuelingDQN, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )
        
        #we make 2 conversion instead of 1 in fully connected layer, one for action advantages, one for state value 
        #prediction
        conv_out_size = self._get_conv_out(input_shape)
        self.fc_adv = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )
        
        self.fc_val = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )
    #we calculate the sample value and action advantage and add them together, then minus the advantage mean and get the 
    #final q-value
    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))
    
    def forward(self, x):
        fx = x.float() / 256
        conv_out = self.conv(fx).view(fx.size()[0], -1)
        val = self.fc_val(conv_out)
        adv = self.fc_adv(conv_out)
        return val + adv - adv.mean()

In [3]:
if __name__ == "__main__":
    #input hyperparameters, check CUDA available, create environment,then we use PTAN DQN wrapper to wrap up the environment
    params = common.HYPERPARAMS['pong']
    parser = argparse.ArgumentParser()
    parser.add_argument("--cuda", default=True, action="store_true", help="Enable cuda")
    args, unknown = parser.parse_known_args()
    device = torch.device("cuda" if args.cuda else "cpu")
    
    env = gym.make(params['env_name'])
    env = ptan.common.wrappers.wrap_dqn(env)
    
    #we make a writer for the environment and action dimension
    writer = SummaryWriter(comment="-" + params['run_name'] + "-dueling")
    net = DuelingDQN(env.observation_space.shape, env.action_space.n).to(device)
    #the wrapper below can create a copy of DQN network, which is target network, and constantly synchronize with online
    #network
    tgt_net = ptan.agent.TargetNet(net)
    
    #we create agent to change observation to action value, we also need action selector to choose the action we use
    #We use epsilon greedy method as action selector here
    selector = ptan.actions.EpsilonGreedyActionSelector(epsilon=params['epsilon_start'])
    epsilon_tracker = common.EpsilonTracker(selector, params)
    agent = ptan.agent.DQNAgent(net, selector, device=device)
    
    #experience source is from one step ExperienceSourceFirstLast and replay buffer, it will store fixed step transitions
    exp_source = ptan.experience.ExperienceSourceFirstLast(env, agent, gamma=params['gamma'], steps_count=1)
    buffer = ptan.experience.ExperienceReplayBuffer(exp_source, buffer_size=params['replay_size'])
    
    #create optimizer and frame counter
    optimizer = optim.Adam(net.parameters(), lr=params['learning_rate'])
    frame_idx = 0
    
    #reward tracker will report mean reward when episode end, and increase frame counter by 1, also getting a transition
    #from frame buffer.
    #buffer.populate(1) will activate following actions:
    #ExperienceReplayBuffer will request for next transition from experience source.
    #Experience source will send the observation to agent to get the action
    #Action selector which use epsilon greedy method will choose an action based on greedy or random
    #Action will be return to experience source and input to the environment to get reward and next observation, 
    # current observation, action, reward, next observation will be stored into replay buffer
    #transfer information will be stored in replay buffer, and oldest observation will be dropped
    with common.RewardTracker(writer, params['stop_reward']) as reward_tracker:
        while True:
            frame_idx += 1
            buffer.populate(1)
            epsilon_tracker.frame(frame_idx)
            
            #check undiscounted reward list after finishing an episode, and send to reward tracker to record the data
            #Maybe it just play one step or didn't have finished episode, if it returns true, it means the mean reward
            #reached the reward boundary and we can break and stop training
            new_rewards = exp_source.pop_total_rewards()
            if new_rewards:
                if reward_tracker.reward(new_rewards[0], frame_idx, selector.epsilon):
                    break
            
            #we check buffer has cached enough data to start training or not. If not, we wait for more data.
            if len(buffer) < params['replay_initial']:
                continue
            
            #here we use Stochastic Gradient Descent(SGD) to calculate loss, zero the gradient,batch from the replay buffer
            optimizer.zero_grad()
            batch = buffer.sample(params['batch_size'])
            loss_v = common.calc_loss_dqn(batch, net, tgt_net.target_model, gamma=params['gamma'], device=device)
            loss_v.backward()
            optimizer.step()
            
            #synchronize the target network with the online network constantly
            if frame_idx % params['target_net_sync'] == 0:
                tgt_net.sync()

859: done 1 games, mean reward -21.000, speed 194.19 f/s, eps 0.99
1798: done 2 games, mean reward -21.000, speed 223.57 f/s, eps 0.98
2559: done 3 games, mean reward -21.000, speed 207.74 f/s, eps 0.97
3379: done 4 games, mean reward -21.000, speed 158.02 f/s, eps 0.97
4477: done 5 games, mean reward -20.800, speed 153.94 f/s, eps 0.96
5512: done 6 games, mean reward -20.500, speed 154.17 f/s, eps 0.94
6425: done 7 games, mean reward -20.429, speed 156.81 f/s, eps 0.94
7419: done 8 games, mean reward -20.375, speed 157.00 f/s, eps 0.93
8624: done 9 games, mean reward -20.111, speed 156.81 f/s, eps 0.91
9446: done 10 games, mean reward -20.200, speed 156.51 f/s, eps 0.91
10265: done 11 games, mean reward -20.273, speed 90.83 f/s, eps 0.90
11087: done 12 games, mean reward -20.333, speed 50.00 f/s, eps 0.89
11850: done 13 games, mean reward -20.385, speed 47.62 f/s, eps 0.88
12700: done 14 games, mean reward -20.429, speed 47.90 f/s, eps 0.87
13530: done 15 games, mean reward -20.467, s

225096: done 119 games, mean reward -12.350, speed 43.57 f/s, eps 0.02
227310: done 120 games, mean reward -11.990, speed 44.01 f/s, eps 0.02
229410: done 121 games, mean reward -11.660, speed 43.25 f/s, eps 0.02
232495: done 122 games, mean reward -11.350, speed 44.64 f/s, eps 0.02
234399: done 123 games, mean reward -10.970, speed 44.60 f/s, eps 0.02
236573: done 124 games, mean reward -10.600, speed 42.77 f/s, eps 0.02
239437: done 125 games, mean reward -10.250, speed 42.88 f/s, eps 0.02
241655: done 126 games, mean reward -9.880, speed 42.25 f/s, eps 0.02
243925: done 127 games, mean reward -9.510, speed 42.10 f/s, eps 0.02
246711: done 128 games, mean reward -9.180, speed 44.20 f/s, eps 0.02
248864: done 129 games, mean reward -8.870, speed 42.94 f/s, eps 0.02
251703: done 130 games, mean reward -8.550, speed 43.82 f/s, eps 0.02
253611: done 131 games, mean reward -8.180, speed 43.81 f/s, eps 0.02
256019: done 132 games, mean reward -7.830, speed 43.19 f/s, eps 0.02
257682: done 